In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
df: pd.DataFrame = pd.read_csv(
    'data/processed_data_cleaned.csv',
    parse_dates=["Time"],
    index_col=[0],
    sep=';'
)

In [25]:
df['time'] = df.index
df.reset_index(drop=True, inplace=True)

In [26]:
def analyze_motor_breaks(df, motor_column='corrente_motor', threshold=1500):
    """
    Analyze breaks in the motor process based on the 'corrente_do_motor' column.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    motor_column (str): The column name for motor current. Default is 'corrente_do_motor'.
    threshold (float): The threshold below which the motor is considered off. Default is 1500.

    Returns:
    pd.DataFrame: A DataFrame with the start and end times of each motor-off period.
    """
    # Create a boolean mask where the motor is off
    motor_off = df[motor_column] < threshold

    # Find the transitions (where the motor state changes)
    transitions = motor_off.ne(motor_off.shift())

    # Assign a group number to each continuous motor-off period
    group_ids = transitions.cumsum()

    # Filter only the motor-off periods
    motor_off_periods = df[motor_off]

    # Group by the group IDs and get the start and end times of each motor-off period
    motor_off_analysis = motor_off_periods.groupby(group_ids[motor_off]).agg(
        start_time=('time', 'min'),
        end_time=('time', 'max')
    ).reset_index(drop=True)

    return motor_off_analysis

In [27]:
df.head()

,feature,alimentação_bu,alimentação_bs,ceee,%_clínquer,%_gesso_mineral,%__gesso_sintético,%_calcário,%_pó_filler,%_escória,...,camada_-_r4,camada_-_r5,camada_-_r6,pressão_-_r1,pressão_-_r2,pressão_-_r3,pressão_-_r4,pressão_-_r5,pressão_-_r6,time
0,NaN,NaN,NaN,14.854868,84.837698,4.510000,0.0,5.641644,4.991417,0.016063,...,62.22339,61.265630,52.756930,0.868055,0.694445,0.347222,0.347222,3.645833,0.868055,2023-08-01 04:54:00
1,NaN,NaN,NaN,18.568586,84.837698,4.510000,0.0,5.641644,4.991417,0.016063,...,62.22339,61.265630,52.756930,0.868055,0.694445,0.347222,0.347222,3.645833,0.868055,2023-08-01 04:55:00
2,NaN,NaN,NaN,8.081791,84.837698,4.513178,0.0,5.641644,4.991417,0.016063,...,80.30786,88.392360,79.522000,154.861100,154.513900,152.777800,153.819400,153.472200,155.555600,2023-08-01 04:56:00
3,NaN,NaN,NaN,12.122686,84.837698,4.513178,0.0,5.641644,4.991417,0.016063,...,80.30786,88.392364,79.522003,154.861115,154.513885,152.777786,153.819443,153.472229,155.555557,2023-08-01 04:57:00
4,0.0431,41.014873,40.956271,23.732629,84.837698,4.513178,0.0,5.641644,4.990000,0.016063,...,80.30786,88.392360,79.522000,154.861100,154.513900,152.777800,153.819400,153.472200,155.555600,2023-08-01 04:58:00


In [ ]:
motor_off_analysis = analyze_motor_breaks(df)

,start_time,end_time
0,2023-08-01 04:54:00,2023-08-01 21:42:00
1,2023-08-02 00:14:00,2023-08-02 01:11:00
2,2023-08-02 01:30:00,2023-08-02 01:48:00
3,2023-08-02 16:53:00,2023-08-02 20:08:00
4,2023-08-03 16:59:00,2023-08-03 20:05:00


In [30]:
motor_off_analysis

,start_time,end_time
0,2023-08-01 04:54:00,2023-08-01 21:42:00
1,2023-08-02 00:14:00,2023-08-02 01:11:00
2,2023-08-02 01:30:00,2023-08-02 01:48:00
3,2023-08-02 16:53:00,2023-08-02 20:08:00
4,2023-08-03 16:59:00,2023-08-03 20:05:00
...,...,...
257,2024-02-28 15:54:00,2024-02-28 21:21:00
258,2024-02-28 21:38:00,2024-03-01 21:03:00
259,2024-03-02 07:10:00,2024-03-02 08:02:00
260,2024-03-03 06:49:00,2024-03-03 09:43:00
